In [1]:
#Build graph
import graph as graph
import random
import myItemClass as mi
import unionFind as uf

# random.seed(500)
mazeHeight= int(input("Enter maze height: "))
mazeWidth= int(input("Enter maze width: "))

uf=uf.UnionFind()
mi=mi.myItems()

arrayOfGraphVertices=[[None]*(mazeWidth) for _ in range(mazeHeight)]
myGraph=graph.Graph()

for j in range(mazeWidth):
    for i in range(mazeHeight):
        u=myGraph.insert_vertex((i,j))
        arrayOfGraphVertices[i][j]=u
        if i>0:
            v=arrayOfGraphVertices[i-1][j]
            myGraph.insert_edge(u,v,random.random())
        if j>0:
            v=arrayOfGraphVertices[i][j-1]
            myGraph.insert_edge(u,v,random.random())

# for edge in myGraph.edges():
#     print(edge)

Enter maze height: 250
Enter maze width: 100


In [2]:
import priorityQueue as pq

mpq=pq.mergeable_priority_queue()

#Debugging purposes
def print_pq(posiItem):
    print("\n",posiItem.element().heap().get_element().edge())
    currentItem=posiItem.element().heap()
    while currentItem.get_leftChild() != None:
        lc=currentItem.get_leftChild().get_element().edge()
        rc="**"
        if currentItem.get_rightChild() != None:
            rc=currentItem.get_rightChild().get_element().edge()
        print(lc,rc)
        currentItem = currentItem.get_leftChild()

#create container items for each vertex and its associated heap of edges
arrayOfPositionItems=[[None]*(mazeWidth) for _ in range(mazeHeight)]
activePositionItems=set()
interiorEdges=[]
for j in range(mazeWidth):
    for i in range(mazeHeight):
        v=arrayOfGraphVertices[i][j]
        candEdgeItemsList=[mi.make_edgeItem(edge.element(),edge) for edge in myGraph.incident_edges(v)]
        candEdgeItems=mpq.make_mpq(candEdgeItemsList.pop())
        for edgeItem in candEdgeItemsList:
            singleton_mpq=mpq.make_mpq(edgeItem)
#             candEdgeItems=mpq.meld(candEdgeItems,singleton_mpq)
            candEdgeItems=candEdgeItems.meld(singleton_mpq)
        vertexItem=mi.make_vertexItem(v,candEdgeItems)
        singleConnectedComponent=uf.make_group(vertexItem)
        arrayOfPositionItems[i][j]=singleConnectedComponent
        activePositionItems.add(singleConnectedComponent)

# for posiItem in activePositionItems:
#     print_pq(posiItem)

In [3]:
def mpq_merge(a,b):
    """Takes vertexContainers a and b
    combines their heaps into a list with no duplicates,
    heapifies the result, and places into a's heap"""
#     a.set_heap(mpq.meld(a.heap(),b.heap()))
    a.set_heap(a.heap().meld(b.heap()))

#Boruvka Sollin algorithm
while len(activePositionItems)>0:
    u_positionItem=activePositionItems.pop()
    u_heap=u_positionItem.element().heap()
    u_parent=uf.find(u_positionItem)
    if u_heap != None:
        min_cand_u, u_heap=mpq.pop_min(u_heap)
        min_cand_edge_u=min_cand_u.edge()
        u=u_positionItem.element().vertex()

        v_row,v_col=min_cand_edge_u.opposite(u).element()
        v_positionItem=arrayOfPositionItems[v_row][v_col]
        v_parent=uf.find(v_positionItem)
        v_heap=v_parent.element().heap()

        min_cand_edge_v=None
        if v_heap != None:
            min_cand_v=v_heap.get_element()
            min_cand_edge_v=min_cand_v.edge()
            v_target=min_cand_edge_v.opposite(v)
        if v_parent is not u_parent:                                    #Diff parents <=> diff connected components
            min_cand_edge=min_cand_edge_u
            if u_parent == v_target and min_cand_edge_v.element() < min_cand_edge_u.element():
                min_cand_v, v_heap=mpq.pop_min(v_heap)
                min_cand_edge=min_cand_v.edge()
            interiorEdges.append(min_cand_edge)                         #Save edge which connects two different connected components
            uf.union(v_parent,u_parent,mpq_merge)                       #Merge connected components. Heaps merged with mpq.meld
            if u_heap != None:
                u_positionItem.element().set_heap(u_heap)
                activePositionItems.add(u_positionItem)                 #Add connected component back to queue
            if v_heap != None:
                v_positionItem.element().set_heap(v_heap)
                activePositionItems.add(v_positionItem)                 #Add connected component back to queue

# for edge in interiorEdges:
#     print(edge)

In [4]:
# Function to find path from v to z via DFS
# Takes:   graph mst G
#          vertex v (start)
#          vertex z (end)
#          dict discovered {vertex: edge}
#          list S (stack of vertices)
#
# Returns: returns boolean for maze found
def pathDFS(G, v, z, discovered, S=[]):
    pathFound = False

    if not pathFound:                                           # Put the root of subtree in stack
        S.append(v)
    if v == z:                                                  # If end vertex v == z, return the stack which is a path
        return True                                             # When G is a MST, S is the shortest path from the start node to z
    else:
        for e in G.incident_edges(v):                           # Iterate through the incident edges of v
            u = e.opposite(v)                                   # Get vertex adjacent to v via e
            #print("vertex: " + str(u.element()) + " edge: " + str((e.endpoints()[0].element(),e.endpoints()[1].element())) +"\n")
            if discovered.get(u) is None:                       # Check if u has been visited
                discovered[u] = e                               # Now we have visited u, so update discovered
                if not pathFound:
                    S.append(e)                                 # Include the edge to specify how we traversed from u to v
                ##TODO: next line redundant
                if not pathFound:                               # Check if we have found the last node
                    pathFound = pathDFS(G, u, z, discovered, S) # Recursively find the path we desire
                if not pathFound:                               # Now we can remove the edge if we did not find the desired path
                    S.pop()
        if not pathFound:                                       # v discovered before and target vertex not in subtree, remove v from stack
            S.pop()                                             # i.e. v is not on the path from the start node to end node
            return False
        else:
            return True

In [5]:
# Function to print maze w/ or w/o solution
# Takes:   graph mst G
#          list of path edges from DFS maze
#          int mazeWidth
#          bool solution
#
# Returns: String to print maze
def print_maze(G, maze, mazeWidth, solution=False):
    multiplier = 2*(mazeWidth + 1)
    # Initialize grid
    topWall="__"*mazeWidth+"_"
    otherWalls = "|_"*mazeWidth+"|"
    mazeRender = topWall + ("\n" + otherWalls)*mazeHeight
    tempList = list(mazeRender)
    # Remove walls and floors for maze
    for e in G.edges():                                    # each edge in MST graph
        u,v  =e.endpoints()
        u_row,u_col=u.element()
        v_row,v_col=v.element()
        if (u_row == v_row):                               #Cross wall: Replace "|" with "_"
            skipTo = (u_row + 1) * multiplier              # +1 off top
            intoRow =u_col + v_col + 1                     # between cells, +1 off left wall
            tempList[skipTo + intoRow] = "_"
        elif (u_col == v_col):                             #Cross floor: Replace "_" with " "
            skipTo = (min(u_row, v_row) + 1) * multiplier  # jump to min row, +1 off top
            intoRow = 2*(u_col + 1)                        # same col, +1 off left wall
            tempList[skipTo + intoRow - 1] = " "
    if solution:                                           # print solution if provided
        """TODO: print start and end cells"""
        for component in maze:                             # iterate through alternating vertex-edge list
            try:                                           # Handle edges
                u,v = component.endpoints()
                u_row,u_col=u.element()
                v_row,v_col=v.element()
                if (u_row == v_row):
                    skipTo = (u_row + 1) * multiplier
                    intoRow = u_col + v_col + 1
                    tempList[skipTo + intoRow] = "*"
                elif (u_col == v_col):
                    skipTo = (min(u_row, v_row) + 1) * multiplier
                    intoRow = 2 * (u_col + 1)
                    tempList[skipTo + intoRow - 1] = "*"
            except:
                u_row, u_col = component.element()         # Handle vertices
                skipTo = (u_row + 1) * multiplier
                intoRow = 2*u_col + 2
                tempList[skipTo + intoRow - 1] = "*"
    mazeRender = "".join(tempList)
    print(mazeRender)

In [6]:
#Initialize new graph object that is MST
arrayOfMazeVertices=[[None]*(mazeWidth) for _ in range(mazeHeight)]
mstMaze=graph.Graph()
for i in range(mazeHeight):
    for j in range(mazeWidth):
        u=mstMaze.insert_vertex((i,j))
        arrayOfMazeVertices[i][j]=u

#Using edges from MST from myGraph, build mstMaze graph
for ind in range(len(interiorEdges)):
    edge   = interiorEdges[ind]                        #grab edge
    u,v    = edge.endpoints()                          #get endpoints u,v
    ux, uy = u.element()                               #get u,v coordinates
    vx, vy = v.element()
    mst_u  = arrayOfMazeVertices[ux][uy]               #get u,v from mstMaze
    mst_v  = arrayOfMazeVertices[vx][vy]
    weight = edge.element()                            #get edge weight
    mstMaze.insert_edge(mst_u,mst_v,weight)            #generate identical edge in mstMaze
    interiorEdges[ind]=mstMaze.get_edge(mst_u,mst_v)   #overwrite edge in list from myGraph with edge from mstMaze

#TODO: specify or random start/end
startVert = arrayOfMazeVertices[0][0]
endVert   = arrayOfMazeVertices[-1][-1]
discovered={startVert:None}
StackOfVertices=[]

solution=pathDFS(mstMaze,startVert,endVert,discovered,StackOfVertices)

print_maze(mstMaze,StackOfVertices,mazeWidth)
print_maze(mstMaze,StackOfVertices,mazeWidth, solution)

_________________________________________________________________________________________________________________________________________________________________________________________________________
| | ___ | ___ _ ___ | _____ _____ |__ _ _ _____ _ |__ __| _______ _________ __________| ____________|______ ___ _______ | _ ___ _ _ _ | _____ |__ |__ ___ _____ | _______ _____ ___ ___ _ _ | _ |______ |
|__ | |__ |__ |_|____ | |___| | __| _ | | | | | |__ ___ ____| | | | |__ _ |__ | _ __| _ | _ | _ |__ | _ | __| _ _ |__ _ | |_| __| |_| |__ | ___ | ____|___| ______|__ |__ |_____|___|__ | |___|_________|
|___| __|__ | |____ |_|____ | | _ |_|_|_|___|_____| | | | | | | | | |__ | | __|_| _ |_|___| _ |_| __|_|___| __|_| | _ | __|___|_____| | __|___|__ | | _ __| | _ | _ |__ |_| | | |__ ____| ___________ __|
| _ | __| __|_|____ | | _ _ | __| __| |__ ___ __| |___| | _ | | __|__ | |_| _ |___| | ____| |__ _ | |____ | ___ |_|_|_|___| _______ __| ____|______ |_| | |___|___|__ |____ _ |____ |__ ___ | _ 